In [4]:
import json
import pandas as pd
import ntpath
from sklearn.preprocessing import MinMaxScaler

# Compare Between PARA and SPAQ

In [8]:
# Compare betweem PARA and SPAQ

para_spaq = open("/media/ian/Backup Drive/Dataset/search_spaq_results.json")
data = json.load(para_spaq)

in_para = 0
in_spaq = 0
shared_image = 0
first_path_sharedlImages = []
second_path_sharedImages = []

i = 0
for key, value in data.items():
    j = 0
    for key2, value2 in value.items():
        if j == 0:
            first_path = value2
            j = 1
        elif j == 1:
            for key3, value3 in value2.items():
                second_path = value3['location']
                # print(first_path, second_path)
                # now compare between the 2 paths 
                if 'PARA' in first_path and 'PARA' in second_path:
                    in_para+=1
                elif 'SPAQ' in first_path and 'SPAQ' in second_path:
                    in_spaq+=1
                else:
                    shared_image+=1
                    # print(first_path, second_path)
                    first_path_sharedlImages.append(first_path)
                    second_path_sharedImages.append(second_path)


print('In PARA duplicated:', in_para)
print('In SPAQ duplicated:', in_spaq)
print('Shared duplicates:', shared_image)
print('Total duplicates:', in_para+in_spaq+shared_image)

In PARA duplicated: 287
In SPAQ duplicated: 323
Shared duplicates: 29
Total duplicates: 639


## Check rating difference between the 2 datasets/same dataset.

### Shared duplicates: 29 for the former 

In [3]:
path_to_PARA_scores = '/media/ian/Backup Drive/FYP stuff/Ian/mtaiq/'
path_to_SPAQ_scores = '/media/ian/Backup Drive/Dataset/SPAQ dataset-20230407T121509Z-008/SPAQ dataset/Annotations/'

# SPAQ_exif = pd.read_excel(path_to_SPAQ_scores + 'EXIF_tags.xlsx')
# SPAQ_cat = pd.read_excel(path_to_SPAQ_scores + 'Scene category labels.xlsx')
SPAQ_score = pd.read_excel(path_to_SPAQ_scores + 'MOS and Image attribute scores.xlsx')
PARA_score = pd.read_csv(path_to_PARA_scores + 'PARA_images_all_image_mean_score.csv')

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

PARA_images = [path_leaf(path) for path in first_path_sharedlImages]
SPAQ_images = [path_leaf(path) for path in second_path_sharedImages]

# create dataframe for mapping the first path and second path images
PARA_SPAQ_merged_dict = {'PARA_sessionId_imageName':PARA_images,'SPAQ_imageName':SPAQ_images}
PARA_SPAQ_merged = pd.DataFrame(PARA_SPAQ_merged_dict)

In [4]:
scaler = MinMaxScaler(feature_range=(1.0, 5.0))
SPAQ_score['normalized_MOS'] = scaler.fit_transform(SPAQ_score['MOS'].values[:, None])
SPAQ_score = SPAQ_score[SPAQ_score['Image name'].isin(SPAQ_images)]
SPAQ_score = SPAQ_score[["Image name", "normalized_MOS"]].reset_index(drop=True)
SPAQ_score.rename(columns={'Image name':'SPAQ_imageName','normalized_MOS':'SPAQ_normalized_MOS'}, inplace=True)
SPAQ_score

,SPAQ_imageName,SPAQ_normalized_MOS
0,00038.jpg,1.255319
1,00303.jpg,1.229787
2,00329.jpg,2.342553
3,00846.jpg,2.966383
4,01343.jpg,1.208936
5,02281.jpg,2.040000
6,02705.jpg,2.384681
7,03339.jpg,1.851064
8,04463.jpg,2.792766
9,04560.jpg,1.294043


In [5]:
PARA_score['sessionId_imageName'] = PARA_score['sessionId'] + '_' + PARA_score['imageName']
PARA_score = PARA_score[PARA_score['sessionId_imageName'].isin(PARA_images)]
PARA_score = PARA_score[["sessionId_imageName", "aestheticScore", "qualityScore"]].reset_index(drop=True)
PARA_score.rename(columns = {'sessionId_imageName':'PARA_sessionId_imageName', 'aestheticScore':'PARA_aestheticScore', 'qualityScore':'PARA_qualityScore'}, inplace=True)
PARA_score

,PARA_sessionId_imageName,PARA_aestheticScore,PARA_qualityScore
0,session125_iaa_pub8718_.jpg,1.660000,1.868000
1,session143_iaa_pub9973_.jpg,1.300000,1.392000
2,session148_iaa_pub10302_.jpg,1.420000,1.584000
3,session158_iaa_pub11049_.jpg,1.903846,2.015385
4,session168_iaa_pub11691_.jpg,1.240000,1.348000
5,session172_iaa_pub12020_.jpg,1.360000,1.416000
6,session188_iaa_pub13152_.jpg,1.444444,1.529630
7,session200_iaa_pub13951_.jpg,1.520000,1.536000
8,session219_iaa_pub15311_.jpg,1.730769,1.780769
9,session227_iaa_pub15888_.jpg,1.480000,1.608000


In [6]:
PARA_SPAQ_merged = pd.merge(PARA_SPAQ_merged, PARA_score, on='PARA_sessionId_imageName')
PARA_SPAQ_merged = pd.merge(PARA_SPAQ_merged, SPAQ_score, on='SPAQ_imageName')
PARA_SPAQ_merged

,PARA_sessionId_imageName,SPAQ_imageName,PARA_aestheticScore,PARA_qualityScore,SPAQ_normalized_MOS
0,session30_iaa_pub2049_.jpg,08024.jpg,1.440000,1.536000,1.808511
1,session59_iaa_pub4107_.jpg,09310.jpg,1.403846,1.473077,1.468085
2,session97_iaa_pub6754_.jpg,10389.jpg,1.460000,1.540000,1.716170
3,session125_iaa_pub8718_.jpg,04463.jpg,1.660000,1.868000,2.792766
4,session143_iaa_pub9973_.jpg,01343.jpg,1.300000,1.392000,1.208936
5,session148_iaa_pub10302_.jpg,06078.jpg,1.420000,1.584000,1.280851
6,session158_iaa_pub11049_.jpg,02705.jpg,1.903846,2.015385,2.384681
7,session168_iaa_pub11691_.jpg,05174.jpg,1.240000,1.348000,1.357447
8,session172_iaa_pub12020_.jpg,07089.jpg,1.360000,1.416000,1.276596
9,session188_iaa_pub13152_.jpg,04560.jpg,1.444444,1.529630,1.294043


In [7]:
corr = PARA_SPAQ_merged['SPAQ_normalized_MOS'].corr(PARA_SPAQ_merged['PARA_qualityScore'])
print("Correlation qualityScore: ", round(corr, 2))

Correlation aestheticsScore:  0.74
Correlation aestheticsScore:  0.78


### Duplicated images within the dataset

In PARA duplicated: 287 / In SPAQ duplicated: 323 for the latter (if lucky get same user for the same image.)

## Checking duplicates in SPAQ


In [10]:
# Compare betweem PARA and SPAQ

para_spaq = open("/media/ian/BackupDrive/Dataset/search_spaq_results.json")
data = json.load(para_spaq)

in_para = 0
in_spaq = 0
shared_image = 0
first_path_sharedImages = []
second_path_sharedImages = []

i = 0
for key, value in data.items():
    j = 0
    for key2, value2 in value.items():
        if j == 0:
            first_path = value2
            j = 1
        elif j == 1:
            for key3, value3 in value2.items():
                second_path = value3['location']
                # print(first_path, second_path)
                # now compare between the 2 paths 
                if 'PARA' in first_path and 'PARA' in second_path:
                    in_para+=1
                elif 'SPAQ' in first_path and 'SPAQ' in second_path:
                    in_spaq+=1
                    first_path_sharedImages.append(first_path)
                    second_path_sharedImages.append(second_path)
                else:
                    shared_image+=1
                    # print(first_path, second_path)


print('In PARA duplicated:', in_para)
print('In SPAQ duplicated:', in_spaq)
print('Shared duplicates:', shared_image)
print('Total duplicates:', in_para+in_spaq+shared_image)

In PARA duplicated: 287
In SPAQ duplicated: 323
Shared duplicates: 29
Total duplicates: 639


In [15]:
import shutil, os

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

SPAQ_images = [path.replace('Backup Drive', 'BackupDrive') for path in first_path_sharedImages]
SPAQ_images = [path.replace('workstation', 'ian') for path in SPAQ_images]
# SPAQ_images = [path_leaf(path) for path in SPAQ_images]
SPAQ_images

for f in SPAQ_images:
    shutil.copy(f, '/media/ian/BackupDrive/Dataset/in_SPAQ_duplicates')



In [17]:
from difPy import dif

search_in_para_duplicates = dif("/media/ian/BackupDrive/Dataset/in_SPAQ_duplicates")

difPy preparing files: [323/323] [100%]
difPy comparing images: [323/323] [100%]
Found 0 pair(s) of duplicate image(s) in 101.4945 seconds.


In [23]:
path_to_SPAQ_scores = '/media/ian/BackupDrive/Dataset/SPAQ dataset-20230407T121509Z-008/SPAQ dataset/Annotations/'

# SPAQ_exif = pd.read_excel(path_to_SPAQ_scores + 'EXIF_tags.xlsx')
# SPAQ_cat = pd.read_excel(path_to_SPAQ_scores + 'Scene category labels.xlsx')
SPAQ_score = pd.read_excel(path_to_SPAQ_scores + 'MOS and Image attribute scores.xlsx')

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

SPAQ_images_first = [path.replace('Backup Drive', 'BackupDrive') for path in first_path_sharedImages]
SPAQ_images_first = [path.replace('workstation', 'ian') for path in SPAQ_images_first]
SPAQ_images_first = [path_leaf(path) for path in SPAQ_images_first]
print(len(SPAQ_images_first))

SPAQ_images_second = [path.replace('Backup Drive', 'BackupDrive') for path in second_path_sharedImages]
SPAQ_images_second = [path.replace('workstation', 'ian') for path in SPAQ_images_second]
SPAQ_images_second = [path_leaf(path) for path in SPAQ_images_second]
print(len(SPAQ_images_second))

# print(SPAQ_images_first)
# print(SPAQ_images_second)

# create dataframe for mapping the first path and second path images
SPAQ_duplicates_merged_dict = {'SPAQ_imageName_first':SPAQ_images_first}
SPAQ_duplicates_first = pd.DataFrame(SPAQ_duplicates_merged_dict)
SPAQ_duplicates_first

SPAQ_duplicates_merged_dict = {'SPAQ_imageName_second':SPAQ_images_second}
SPAQ_duplicates_second = pd.DataFrame(SPAQ_duplicates_merged_dict)
SPAQ_duplicates_second

# create dataframe for mapping the first path and second path images
SPAQ_duplicates_merged_dict = {'SPAQ_imageName_first':SPAQ_duplicates_first['SPAQ_imageName_first'].to_list(),'SPAQ_imageName_second':SPAQ_duplicates_second['SPAQ_imageName_second'].to_list()}
SPAQ_duplicates = pd.DataFrame(SPAQ_duplicates_merged_dict)
SPAQ_duplicates

323
323


,SPAQ_imageName_first,SPAQ_imageName_second
0,00098.jpg,10298.jpg
1,00107.jpg,10147.jpg
2,00219.jpg,10042.jpg
3,00248.jpg,10871.jpg
4,00275.jpg,10479.jpg
...,...,...
318,09913.jpg,11047.jpg
319,09963.jpg,10691.jpg
320,09982.jpg,10360.jpg
321,09993.jpg,10550.jpg


In [25]:
path_to_SPAQ_scores = '/media/ian/BackupDrive/Dataset/SPAQ dataset-20230407T121509Z-008/SPAQ dataset/Annotations/'

SPAQ_score = pd.read_excel(path_to_SPAQ_scores + 'MOS and Image attribute scores.xlsx')

scaler = MinMaxScaler(feature_range=(1.0, 5.0))
SPAQ_score['normalized_MOS'] = scaler.fit_transform(SPAQ_score['MOS'].values[:, None])

SPAQ_score_first = SPAQ_score[SPAQ_score['Image name'].isin(SPAQ_duplicates_first['SPAQ_imageName_first'].to_list())]
SPAQ_score_first = SPAQ_score_first[["Image name", "normalized_MOS"]].reset_index(drop=True)
SPAQ_score_first.rename(columns={'Image name':'SPAQ_imageName_first','normalized_MOS':'SPAQ_normalized_MOS_first'}, inplace=True)
SPAQ_score_first

,SPAQ_imageName_first,SPAQ_normalized_MOS_first
0,00098.jpg,3.005957
1,00107.jpg,2.569362
2,00219.jpg,3.528511
3,00248.jpg,2.351915
4,00275.jpg,3.563830
...,...,...
318,09963.jpg,2.134894
319,09982.jpg,2.085106
320,09993.jpg,2.106383
321,09995.jpg,3.794468


In [26]:
scaler = MinMaxScaler(feature_range=(1.0, 5.0))
SPAQ_score['normalized_MOS'] = scaler.fit_transform(SPAQ_score['MOS'].values[:, None])
SPAQ_score_second = SPAQ_score[SPAQ_score['Image name'].isin(SPAQ_duplicates_second['SPAQ_imageName_second'].to_list())]
SPAQ_score_second = SPAQ_score_second[["Image name", "normalized_MOS"]].reset_index(drop=True)
SPAQ_score_second.rename(columns={'Image name':'SPAQ_imageName_second','normalized_MOS':'SPAQ_normalized_MOS_second'}, inplace=True)
SPAQ_score_second

,SPAQ_imageName_second,SPAQ_normalized_MOS_second
0,05056.jpg,4.000000
1,06201.jpg,2.994894
2,10004.jpg,2.390213
3,10007.jpg,4.512340
4,10008.jpg,3.540851
...,...,...
318,11111.jpg,4.021277
319,11112.jpg,4.283830
320,11120.jpg,4.390213
321,11121.jpg,2.113617


In [27]:
SPAQ_duplicates

,SPAQ_imageName_first,SPAQ_imageName_second
0,00098.jpg,10298.jpg
1,00107.jpg,10147.jpg
2,00219.jpg,10042.jpg
3,00248.jpg,10871.jpg
4,00275.jpg,10479.jpg
...,...,...
318,09913.jpg,11047.jpg
319,09963.jpg,10691.jpg
320,09982.jpg,10360.jpg
321,09993.jpg,10550.jpg


In [30]:
SPAQ_duplicates_mergedDf = pd.merge(SPAQ_duplicates, SPAQ_score_first, on='SPAQ_imageName_first')
SPAQ_duplicates_mergedDf = pd.merge(SPAQ_duplicates_mergedDf, SPAQ_score_second, on='SPAQ_imageName_second')
# SPAQ_duplicates_mergedDf = SPAQ_duplicates_mergedDf.reindex(columns=['PARA_sessionId_imageName_first', 'PARA_aestheticScore_first', 'PARA_qualityScore_first', 'PARA_sessionId_imageName_second', 'PARA_aestheticScore_second', 'PARA_qualityScore_second'])
SPAQ_duplicates_mergedDf

,SPAQ_imageName_first,SPAQ_imageName_second,SPAQ_normalized_MOS_first,SPAQ_normalized_MOS_second
0,00098.jpg,10298.jpg,3.005957,3.237021
1,00107.jpg,10147.jpg,2.569362,3.111915
2,00219.jpg,10042.jpg,3.528511,4.188085
3,00248.jpg,10871.jpg,2.351915,2.404255
4,00275.jpg,10479.jpg,3.563830,4.170213
...,...,...,...,...
318,09913.jpg,11047.jpg,2.297872,2.354468
319,09963.jpg,10691.jpg,2.134894,2.154894
320,09982.jpg,10360.jpg,2.085106,2.778723
321,09993.jpg,10550.jpg,2.106383,2.418298


In [31]:
corr = SPAQ_duplicates_mergedDf['SPAQ_normalized_MOS_first'].corr(SPAQ_duplicates_mergedDf['SPAQ_normalized_MOS_second'])
print("Correlation score: ", round(corr, 2))

Correlation score:  0.9
